# Spange-Only Features Experiment

**HYPOTHESIS**: Public kernels ("mixall", "best-work-here") use only Spange descriptors (13 features) and achieve competitive results. Our 140-feature approach (Spange + DRFP + ACS) may be overfitting.

**Current Status**:
- Best CV: 0.0083 (exp_030)
- Best LB: 0.0877 (exp_030)
- Target: 0.0730
- CV-LB relationship: LB = 4.23 * CV + 0.0533
- Required CV for target: 0.00466 (43.9% improvement needed)

**Implementation**:
- Use ONLY Spange descriptors (13 features) + Arrhenius kinetics (5 features) = 18 features
- Best ensemble: GP(0.15) + MLP(0.55) + LGBM(0.3)
- Linear mixing for mixtures (no interaction terms)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = (X["SOLVENT A NAME"] != row["SOLVENT A NAME"]) | (X["SOLVENT B NAME"] != row["SOLVENT B NAME"])
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print("Data loading functions defined")

Data loading functions defined


In [3]:
# Load ONLY Spange descriptors (13 features)
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
print(f'Spange descriptors: {SPANGE_DF.shape}')
print(f'Columns: {list(SPANGE_DF.columns)}')

Spange descriptors: (26, 13)
Columns: ['dielectric constant', 'ET(30)', 'alpha', 'beta', 'pi*', 'SA', 'SB', 'SP', 'SdP', 'N', 'n', 'f(n)', 'delta']


In [4]:
# SPANGE-ONLY FEATURIZER - THE KEY SIMPLIFICATION
# Only 18 features: 5 kinetics + 13 Spange

class SpangeOnlyFeaturizer:
    """Featurizer using ONLY Spange descriptors (13 features) + Arrhenius kinetics (5 features)"""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 5 + 13  # kinetics(5) + spange(13) = 18 features

    def featurize(self, X, flip=False):
        # Arrhenius kinetics features
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B = self.spange_df.loc[X["SOLVENT B NAME"]].values
            r = X["SolventB%"].values.reshape(-1, 1) / 100.0
            if flip:
                r = 1 - r
                A, B = B, A
            # Linear mixing only - NO interaction terms
            X_spange = A * (1 - r) + B * r
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip), dtype=torch.double)

print(f'SpangeOnlyFeaturizer defined')
print(f'Feature dimension: {SpangeOnlyFeaturizer().feats_dim} (vs 145 in previous experiments)')

SpangeOnlyFeaturizer defined
Feature dimension: 18 (vs 145 in previous experiments)


In [5]:
# Gaussian Process Wrapper with Spange-Only Features
class GPWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = SpangeOnlyFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scaler = None

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=3, random_state=42)
            gp.fit(X_scaled, y_all[:, i])
            self.models.append(gp)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_std)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_scaled)
            preds.append(pred)
        
        return torch.tensor(np.column_stack(preds), dtype=torch.double)

print('GPWrapper defined with Spange-only features')

GPWrapper defined with Spange-only features


In [6]:
# Weighted Huber Loss
class WeightedHuberLoss(nn.Module):
    def __init__(self, weights=[1.0, 1.0, 2.0]):
        super().__init__()
        self.weights = torch.tensor(weights, dtype=torch.double)
        self.huber = nn.HuberLoss(reduction='none')
    
    def forward(self, pred, target):
        huber_loss = self.huber(pred, target)
        weighted_loss = huber_loss * self.weights.to(pred.device)
        return weighted_loss.mean()

# MLP Model - slightly deeper for simpler features
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32], output_dim=3, dropout=0.1):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLP components defined')

MLP components defined


In [7]:
# MLP Ensemble with Spange-Only Features
class SpangeMLPEnsemble:
    def __init__(self, hidden_dims=[64, 32], n_models=5, data='single', loss_weights=[1.0, 1.0, 2.0]):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.loss_weights = loss_weights
        self.featurizer = SpangeOnlyFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scalers = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        self.scalers = []
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            scaler = StandardScaler()
            X_scaled = torch.tensor(scaler.fit_transform(X_all.numpy()), dtype=torch.double)
            self.scalers.append(scaler)
            
            model = MLPModelInternal(input_dim, self.hidden_dims, dropout=0.1).double().to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
            criterion = WeightedHuberLoss(self.loss_weights)
            
            dataset = TensorDataset(X_scaled, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                epoch_loss = 0
                for X_batch, y_batch in loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = criterion(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                scheduler.step(epoch_loss)
            
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize_torch(X_test, flip=False)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_test, flip=True)
            preds = []
            for model, scaler in zip(self.models, self.scalers):
                model.eval()
                with torch.no_grad():
                    X_std_scaled = torch.tensor(scaler.transform(X_std.numpy()), dtype=torch.double).to(device)
                    X_flip_scaled = torch.tensor(scaler.transform(X_flip.numpy()), dtype=torch.double).to(device)
                    pred_std = model(X_std_scaled)
                    pred_flip = model(X_flip_scaled)
                    pred = (pred_std + pred_flip) / 2  # TTA
                    preds.append(pred.cpu())
            return torch.stack(preds).mean(dim=0)
        else:
            preds = []
            for model, scaler in zip(self.models, self.scalers):
                model.eval()
                with torch.no_grad():
                    X_scaled = torch.tensor(scaler.transform(X_std.numpy()), dtype=torch.double).to(device)
                    pred = model(X_scaled)
                    preds.append(pred.cpu())
            return torch.stack(preds).mean(dim=0)

print('SpangeMLPEnsemble defined with Spange-only features')

SpangeMLPEnsemble defined with Spange-only features


In [8]:
# LightGBM Wrapper with Spange-Only Features
class SpangeLGBMWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = SpangeOnlyFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scaler = None

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        
        for i in range(3):
            train_data = lgb.Dataset(X_scaled, label=y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=500)
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_std)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_scaled)
            preds.append(pred)
        
        return torch.tensor(np.column_stack(preds), dtype=torch.double)

print('SpangeLGBMWrapper defined with Spange-only features')

SpangeLGBMWrapper defined with Spange-only features


In [9]:
# GP + MLP + LGBM Ensemble with Spange-Only Features
class SpangeGPMLPLGBMEnsemble:
    def __init__(self, data='single'):
        self.data_type = data
        self.gp = GPWrapper(data=data)
        self.mlp = SpangeMLPEnsemble(hidden_dims=[64, 32], n_models=5, data=data, loss_weights=[1.0, 1.0, 2.0])
        self.lgbm = SpangeLGBMWrapper(data=data)
        # Best weights from exp_035: GP 0.15, MLP 0.55, LGBM 0.3
        self.weights = {'gp': 0.15, 'mlp': 0.55, 'lgbm': 0.3}

    def train_model(self, X_train, y_train):
        self.gp.train_model(X_train, y_train)
        self.mlp.train_model(X_train, y_train)
        self.lgbm.train_model(X_train, y_train)

    def predict(self, X_test):
        gp_pred = self.gp.predict(X_test)
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        combined = (self.weights['gp'] * gp_pred + 
                    self.weights['mlp'] * mlp_pred + 
                    self.weights['lgbm'] * lgbm_pred)
        return torch.clamp(combined, 0, 1)

print('SpangeGPMLPLGBMEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3)')
print('All models use Spange-only features (18 features vs 145 in previous experiments)')

SpangeGPMLPLGBMEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3)
All models use Spange-only features (18 features vs 145 in previous experiments)


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = SpangeGPMLPLGBMEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save fold predictions
submission_single_solvent = pd.DataFrame(all_predictions)
print(f"Single solvent predictions: {len(submission_single_solvent)} rows")

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = SpangeGPMLPLGBMEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save fold predictions
submission_full_data = pd.DataFrame(all_predictions)
print(f"Full data predictions: {len(submission_full_data)} rows")

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)

# Weighted average (same as competition)
n_single = len(actuals_single)
n_full = len(actuals_full)
mse_overall = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f"\n=== CV SCORE VERIFICATION (Spange-Only Features) ===")
print(f"Single Solvent MSE: {mse_single:.6f} (n={n_single})")
print(f"Full Data MSE: {mse_full:.6f} (n={n_full})")
print(f"Overall MSE: {mse_overall:.6f}")

print(f"\n=== COMPARISON ===")
print(f"exp_035 (best CV, 145 features): CV 0.008194")
print(f"This (Spange-only, 18 features): CV {mse_overall:.6f}")

if mse_overall < 0.008194:
    print(f"\n\u2713 IMPROVEMENT: {(0.008194 - mse_overall) / 0.008194 * 100:.2f}% better than exp_035!")
else:
    print(f"\n\u2717 WORSE: {(mse_overall - 0.008194) / 0.008194 * 100:.2f}% worse than exp_035")

print(f"\n=== CV-LB RELATIONSHIP ANALYSIS ===")
print(f"If CV-LB relationship is LB = 4.23*CV + 0.0533:")
predicted_lb = 4.23 * mse_overall + 0.0533
print(f"Predicted LB: {predicted_lb:.4f}")
print(f"Best LB so far: 0.0877")
print(f"Target LB: 0.0730")

print(f"\n=== KEY INSIGHT ===")
print(f"Feature reduction: 145 -> 18 features (87.6% reduction)")
print(f"If CV improves, simpler features generalize better.")
print(f"If CV is similar, we can use simpler features without loss.")